In [1]:
!pip install langgraph langchain langchain-community langchain-openai openai chromadb tiktoken unstructured serpapi
!pip install chromadb llama-index llama-index-vector-stores-chroma fastembed transformers huggingface_hub torch gradio
!pip install llama-index-embeddings-fastembed
!pip install smolagents torch
!pip install pymongo dnspython google-search-results
!pip install -U llama-index
!pip install chromadb \
    llama-index \
    llama-index-vector-stores-chroma \
    llama-index-embeddings-fastembed \
    llama-index-llms-huggingface \
    fastembed \
    transformers \
    huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 36.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from smolagents import CodeAgent, GoogleSearchTool, HfApiModel, VisitWebpageTool, CodeAgent
from typing import Dict, TypedDict, List
from langgraph.graph import END, StateGraph
from llama_index.core import QueryBundle, VectorStoreIndex, StorageContext, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.tools import FunctionTool
from llama_index.core.agent.workflow import AgentWorkflow, ReActAgent
import chromadb
import torch
from transformers import AutoTokenizer
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from smolagents import GoogleSearchTool
import os
from pymongo import MongoClient
from datetime import datetime
import uuid
from urllib.parse import quote_plus
import json  # Added missing import
from chromadb.utils import embedding_functions
from llama_index.core.schema import TextNode

import numpy as np
import torch
from typing import TypedDict, Optional, List, Dict, Any
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from diffusers import StableDiffusionPipeline

In [3]:
 # Authenticate Hugging Face (if needed)
from huggingface_hub import notebook_login
notebook_login()

In [4]:
# Initialize components
os.environ["SERPER_API_KEY"] = "2b31aea7a2305eaf1d86843679d302b736f3556d"
openrouter_api_key = "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"

In [5]:
chat_model = ChatOpenAI(
    temperature=0,
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=openrouter_api_key,  # Ensure this is the actual key, not a string reference
    model="mistralai/mistral-7b-instruct:v0.1"  # Or any other OpenRouter model
)


sd_pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
sd_pipe.to("cuda")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


scheduler_config-checkpoint.json:   0%|          | 0.00/209 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.32.2",
  "_name_or_path": "CompVis/stable-diffusion-v1-4",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [6]:
# Define all states
class RouterState(TypedDict):
    user_input: str
    request_type: Optional[str]
    messages: List[Dict[str, Any]]

class AgentState(TypedDict):
    question: str
    generation: str
    source: str
    decision_reason: str

class CombinedState(TypedDict):
    user_input: str
    request_type: Optional[str]
    messages: List[Dict[str, Any]]
    question: str
    generation: str
    source: str
    decision_reason: str

In [7]:
# Initialize LLMs
chat_model = ChatOpenAI(
    temperature=0,
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=openrouter_api_key,
    model="mistralai/mistral-7b-instruct:v0.1"
)

# Initialize RAG components
documents = SimpleDirectoryReader("/content/Data").load_data()
embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
stopping_ids = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("</s>"),
]

llm = HuggingFaceLLM(
    context_window=8192,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": True},
    system_prompt="You are a Q&A assistant. Your goal is to answer questions accurately based on the context provided.",
    query_wrapper_prompt="[INST] {query_str} [/INST]",
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    device_map="auto",
    stopping_ids=stopping_ids,
    tokenizer_kwargs={"max_length": 4096},
    model_kwargs={"torch_dtype": torch.float16}
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [8]:
# Initialize ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")
vector_store = ChromaVectorStore(chroma_collection=chroma_client.get_or_create_collection("my_collection"))
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    embed_model=embed_model
)

# Initialize Memory
memory_client = chromadb.PersistentClient(path="./memory_db")
memory_vector_store = ChromaVectorStore(
    chroma_collection=memory_client.get_or_create_collection("conversation_memory")
)
memory_storage_context = StorageContext.from_defaults(vector_store=memory_vector_store)


In [9]:
# Search Tools
def search_web(query: str) -> str:
    try:
        search_tool = GoogleSearchTool(provider="serper")
        results = search_tool(query)
        if isinstance(results, str):
            return results
        formatted = []
        for res in results[:3]:
            title = res.get('title', 'No title')
            link = res.get('link', 'No link')
            snippet = res.get('snippet', 'No snippet')
            formatted.append(f"Title: {title}\nLink: {link}\nSnippet: {snippet}")
        return "\n\n".join(formatted) if formatted else "No web results found"
    except Exception as e:
        return f"Web search error: {str(e)}"

def search_documents(query: str) -> str:
    try:
        query_engine = index.as_query_engine(llm=llm)
        response = query_engine.query(query)
        return str(response) if response else "No relevant documents found"
    except Exception as e:
        return f"Document search error: {str(e)}"


In [20]:
# Memory Functions
def check_memory(state: CombinedState) -> Dict:
    try:
        collection = memory_vector_store._collection
        embedding = embed_model.get_text_embedding(state["question"])
        results = collection.query(
            query_embeddings=[embedding],
            n_results=1,
            include=["documents", "metadatas", "distances"]
        )
        if results and results['distances'][0]:
            similarity_score = 1 - results['distances'][0][0]
            if similarity_score > 0.7:
                return {
                    "generation": results['documents'][0][0],
                    "source": "memory",
                    "decision_reason": f"Memory match (similarity: {similarity_score:.2f})",
                    "question": state["question"]
                }
    except Exception as e:
        print(f"Memory check error: {str(e)}")
    return state

def store_in_memory(state: CombinedState) -> Dict:
    if state["source"] in ["documents", "web"]:
        try:
            chroma_collection = memory_vector_store._collection
            embedding = embed_model.get_text_embedding(state["question"])
            metadata = {
                "question": state["question"],
                "source": state["source"],
                "timestamp": datetime.now().isoformat()
            }
            chroma_collection.add(
                embeddings=[embedding],
                documents=[state["generation"]],
                metadatas=[metadata],
                ids=[str(uuid.uuid4())]
            )
        except Exception as e:
            print(f"Memory storage error: {str(e)}")
    return state

In [11]:
# Decision Agent
class ToolDecisionAgent:
    def __init__(self, llm):
        self.llm = llm
        self.rules = """Decision Rules:
        1. Use documents for relationship marketing questions
        2. Use web for current events/trends
        3. Default to documents if available"""

    def decide_tool(self, question: str) -> Dict:
        prompt = f"""Analyze this question and follow these rules:
        {self.rules}
        Question: {question}
        Return JSON: {{"tool": "documents"|"web", "reason": "explanation"}}"""
        try:
            response = self.llm.complete(prompt)
            return json.loads(response.text)
        except:
            return {"tool": "documents", "reason": "default fallback"}

def decide_tool(state: CombinedState) -> Dict:
    decision_agent = ToolDecisionAgent(llm)
    decision = decision_agent.decide_tool(state["question"])
    return {
        **state,
        "source": decision["tool"],
        "decision_reason": decision["reason"]
    }

def execute_search(state: CombinedState) -> Dict:
    try:
        if state["source"] == "documents":
            result = search_documents(state["question"])
            if "no relevant" in result.lower():
                result = search_web(state["question"])
                state["source"] = "web"
        else:
            result = search_web(state["question"])
        return {**state, "generation": result}
    except Exception as e:
        return {**state, "generation": f"Search error: {str(e)}"}


In [13]:
# Workflows
# RAG Workflow
rag_workflow = StateGraph(AgentState)
rag_workflow.add_node("check_memory", check_memory)
rag_workflow.add_node("decide_tool", decide_tool)
rag_workflow.add_node("execute_search", execute_search)
rag_workflow.add_node("store_memory", store_in_memory)
rag_workflow.set_entry_point("check_memory")
rag_workflow.add_conditional_edges(
    "check_memory",
    lambda x: "store_memory" if x.get("generation") else "decide_tool"
)
rag_workflow.add_edge("decide_tool", "execute_search")
rag_workflow.add_edge("execute_search", "store_memory")
rag_workflow.add_edge("store_memory", END)
rag_app = rag_workflow.compile()

# Router Workflow
router_workflow = StateGraph(RouterState)
router_workflow.add_node("classify_request", classify_request)
router_workflow.set_entry_point("classify_request")
router_workflow.add_conditional_edges(
    "classify_request",
    lambda x: x["request_type"],
    {"text": END, "image": END, "both": END}
)
router_app = router_workflow.compile()


In [22]:
def classify_request(state: RouterState):
    """Improved classification with bulletproof response handling"""
    prompt = """Classify this marketing request. Respond ONLY with:
    - "text" for content requests
    - "image" for visual requests
    - "both" for combined requests

    Input: {input}""".format(input=state['user_input'])

    try:
        response = chat_model.invoke([HumanMessage(content=prompt)])
        # Robust cleaning of response
        clean_response = (
            response.content.strip()
            .lower()
            .replace('"', '')
            .replace("'", "")
            .replace(".", "")
            .strip()
        )

        # Validate against allowed values
        if clean_response not in ["text", "image", "both"]:
            print(f"Invalid classification: {clean_response}")
            clean_response = "text"  # Safe default

        return {
            "request_type": clean_response,
            "messages": state["messages"] + [
                {"role": "system", "content": "Classification successful"},
                {"role": "assistant", "content": clean_response}
            ]
        }
    except Exception as e:
        print(f"Classification failed: {str(e)}")
        return {
            "request_type": "text",
            "messages": state["messages"] + [
                {"role": "system", "content": "Classification failed, defaulting to text"}
            ]
        }

def run_agent():
    print("Marketing AI Assistant Initialized")
    print(f"Documents: {'Loaded' if documents else 'Not available'}")

    while True:
        try:
            user_input = input("\nQuestion (or 'exit' to quit): ").strip()

            if user_input.lower() in ['exit', 'quit', 'bye']:
                print("Goodbye!")
                break

            if not user_input:
                continue

            # First try memory
            memory_response = check_memory({
                "question": user_input,
                "generation": "",
                "source": "",
                "decision_reason": "",
                "user_input": user_input,
                "request_type": None,
                "messages": []
            })

            if memory_response.get("generation"):
                print(f"\nAnswer (from memory):\n{memory_response['generation']}")
                continue

            # Classify request with error handling
            try:
                classification = classify_request({
                    "user_input": user_input,
                    "request_type": None,
                    "messages": []
                })
                req_type = classification["request_type"]
            except:
                req_type = "text"

            # Handle each request type with fallbacks
            if req_type == "text":
                try:
                    result = rag_app.invoke({
                        "question": user_input,
                        "generation": "",
                        "source": "",
                        "decision_reason": ""
                    })
                    print(f"\nMarketing Answer:\n{result['generation']}")
                except:
                    print("\nHere's information about marketing strategies...")

            elif req_type == "image":
                print("\nFor marketing visuals, I recommend:")
                print("- Canva: canva.com")
                print("- Adobe Express: adobe.com/express")

            elif req_type == "both":
                try:
                    result = rag_app.invoke({
                        "question": user_input,
                        "generation": "",
                        "source": "",
                        "decision_reason": ""
                    })
                    print(f"\nMarketing Content:\n{result['generation']}")
                    print("\nVisual Suggestions:")
                    print("- Create an infographic with key points")
                    print("- Design social media posts")
                except:
                    print("\nMarketing advice with visual recommendations...")

        except Exception as e:
            print("\nLet me provide some general marketing guidance...")
            print("Relationship marketing focuses on customer retention...")

if __name__ == "__main__":
    run_agent()

Marketing AI Assistant Initialized
Documents: Loaded

Question (or 'exit' to quit): What is relation marketing?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Marketing Answer:
Relationship marketing is a marketing strategy that focuses on building long-term engagement and loyalty with customers, rather than just acquiring new ones. It emphasizes the importance of personalized experiences, loyalty programs, excellent customer service, and consistent engagement in creating customer relationships. Relationship marketing differs from traditional marketing in that it focuses on customer retention instead of customer acquisition. Industries that benefit most from relationship marketing include e-commerce, healthcare, retail, banking, and service-based industries.

Question (or 'exit' to quit): What is relation marketing?

Answer (from memory):
Relationship marketing is a marketing strategy that focuses on building long-term engagement and loyalty with customers, rather than just acquiring new ones. It emphasizes the importance of personalized experiences, loyalty programs, excellent customer service, and consistent engagement in creating custome

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Marketing Content:
Sure! Here is an infographic to promote your coffee shop:

[Insert image of coffee shop here]

Infographic:

[Insert image of coffee shop here]

Title: Visit our Coffee Shop

Subtitle: Enjoy the Best Coffee in Town

Bullet points:

* Our coffee is made from the finest beans
* We have a wide selection of coffee drinks to choose from
* Our friendly staff will make you feel welcome
* We have a comfortable and cozy atmosphere
* Visit us today and experience the best coffee in town!

Call to action: Visit our Coffee Shop Now!

Visual Suggestions:
- Create an infographic with key points
- Design social media posts

Question (or 'exit' to quit): suggest me some inflencer instagram usernames to help me promote my coffee shop online.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Marketing Answer:
Influencer Instagram usernames to promote your coffee shop online:

1. @coffee\_lover\_1
2. @coffee\_addict
3. @caffeine\_craver
4. @coffee\_buddies
5. @coffee\_enthusiast
6. @coffee\_connoisseur
7. @coffee\_shop\_gram
8. @coffee\_lifestyle
9. @coffee\_culture
10. @coffee\_artisan

You can also search for hashtags like #coffee, #coffeeaddict, #coffeehunt, #coffeeobsessed, and #coffeeshop to find more influencer accounts. You can also consider partnering with micro-influencers who have a smaller but more engaged following.

Question (or 'exit' to quit): make an infographic to promote my coffee shop

Answer (from memory):
Sure! Here is an infographic to promote your coffee shop:

[Insert image of coffee shop here]

Infographic:

[Insert image of coffee shop here]

Title: Visit our Coffee Shop

Subtitle: Enjoy the Best Coffee in Town

Bullet points:

* Our coffee is made from the finest beans
* We have a wide selection of coffee drinks to choose from
* Our friendly staf

In [23]:
def debug_memory():
    """Print current memory contents"""
    collection = memory_vector_store._collection
    results = collection.peek()
    print("\nMemory Contents:")
    for doc, meta in zip(results['documents'], results['metadatas']):
        print(f"Question: {meta['question']}")
        print(f"Answer: {doc[:100]}...")
        print("-----")

In [24]:
debug_memory()


Memory Contents:
Question: What is relation marketing?
Answer: Relationship marketing is a marketing strategy that focuses on building long-term engagement and loy...
-----
Question: make an infographic to promote my coffee shop
Answer: Sure! Here is an infographic to promote your coffee shop:

[Insert image of coffee shop here]

Infog...
-----
Question: suggest me some inflencer instagram usernames to help me promote my coffee shop online.
Answer: Influencer Instagram usernames to promote your coffee shop online:

1. @coffee\_lover\_1
2. @coffee\...
-----
